# 10Pearls - User Profile Creation Tool 
---

In [ ]:
pip install -r requirements.txt

In [2]:
import os
import tempfile
from docx.shared import Pt
from openai import Client
from docx import Document
from docx.enum.section import WD_SECTION
from dotenv import load_dotenv
from python_docx_replace import docx_replace
import re

### Uploading the File
Keep in mind this works exclusively with old profile formats and not CVs

In [3]:
load_dotenv()
key = os.getenv("OPENAI_API_KEY")

In [54]:
old_doc_path = os.path.join('samples', 'Munazza Jamal - QA Manual.docx')
old_doc = Document(old_doc_path)

old_doc_text = ""

for paragraph in old_doc.paragraphs:
    old_doc_text += paragraph.text + "\n"


#print(old_doc_text)

In [55]:
job_description = "They have a need of at least 2 to 3 Data Engineers with solid experience in Databricks, and creating and manipulating ETL jobs. Databricks certification will be ideal."

### Passing text through OpenAI API 

In [56]:
client = Client(api_key=key)

prompt = f"""Here is the CV/Resume of a person, I want you to efficiently convert the following profile into the new format provided below .
I also need you to compare the following job decription (which is basically a requirement or need for a person) with the person's profile and give me a percentage of how likely the person
is to be a match for the job. I also want you to give me the missing keywords that are present in the job description but not in the person's profile  
Ensure the output is consistently structured as follows:

Job Description:
{job_description}

Old format:
{old_doc_text}

New format:

Resource Name: (Extract and display the profile's Resource Name as the heading)
Percentage Match: (Percentage of how much the person matches the job)
Missing Keywords: (Keywords that are present in job description but not in users profile)
Job Role: (Extract and display the profile's Job Role from the summary as the heading)
Summary: (Keep the summary the same)
Key Skills & Competencies: (Identify and extract at least 6 attributes from the profile and explain them. Examples include "technical expertise", "technical toolkit", "communication and collaboration")
Academic Summary: (Display as provided)
Work Summary: (Provide in detailed paragraphs without altering any information. For each project, include the following details)
   - Project 1, 2 or whatever: 
   - Environment: (Always identify and display the job role from the respective project)
   - Project Name: (Display under the "Project Name" heading)
   - Outline and Responsibilities: (For each project, detail the outline and responsibilities)


                     """



response = client.chat.completions.create(
    model='gpt-3.5-turbo-16k',
    messages=[{"role": "assistant", "content": [{"type": "text", "text": prompt}]}],
    max_tokens=2000,
    temperature=0.2,
    top_p=1.0
)

In [57]:
resp = response.choices[0].message.content
print(resp)

Resource Name: Munazza Jamal
Percentage Match: 40%
Missing Keywords: Data Engineers, Databricks certification
Job Role: Manual and Automation Software Tester
Summary: A highly motivated resource with 5+ years of experience in the field of Manual and Automation software testing. Understands and possesses test management skills as well as automation scripting skills under the best tools and frameworks. Skilled in Smoke Testing, Functional Testing, Integration Testing, System Testing, User Interface testing etc. Have proven expertise in working on different types of test deliverables – Test Plans, Test Cases, Testing Strategies, Test scripts and Test summary reports. Very well acquainted with Test Management and Defect tracking systems such as JIRA and Azure DevOps. Experience in leading, mentoring and guiding the QA team. Also proficient in Test Automation tools such as Selenium WebDriver, Appium, and Cypress.

Key Skills & Competencies:
1. Technical Expertise: Munazza has 5+ years of ex

### Processing the response 
Removing additional line spaces so that response is always in a fixed format

In [58]:
resp = resp.split("\n")

In [38]:
resp[2]

''

In [59]:
resp.remove('')

In [60]:
print(resp)

['Resource Name: Munazza Jamal', 'Percentage Match: 40%', 'Missing Keywords: Data Engineers, Databricks certification', 'Job Role: Manual and Automation Software Tester', 'Summary: A highly motivated resource with 5+ years of experience in the field of Manual and Automation software testing. Understands and possesses test management skills as well as automation scripting skills under the best tools and frameworks. Skilled in Smoke Testing, Functional Testing, Integration Testing, System Testing, User Interface testing etc. Have proven expertise in working on different types of test deliverables – Test Plans, Test Cases, Testing Strategies, Test scripts and Test summary reports. Very well acquainted with Test Management and Defect tracking systems such as JIRA and Azure DevOps. Experience in leading, mentoring and guiding the QA team. Also proficient in Test Automation tools such as Selenium WebDriver, Appium, and Cypress.', 'Key Skills & Competencies:', '1. Technical Expertise: Munazza

In [61]:
text = "\n".join(resp)

# Use regex to extract the required sections
resource_name = re.search(r"Resource Name:\s*(.*)", text).group(1)
job_role = re.search(r"Job Role:\s*(.*)", text).group(1)
percentage_match = re.search(r"Percentage Match:\s*(.*)", text).group(1)
missing_keywords = re.search(r"Missing Keywords:\s*(.*)", text).group(1)
summary = re.search(r"Summary:\s*(.*)", text).group(1)
key_skills_competencies = re.search(r"Key Skills & Competencies:\s*(.*?)Academic Summary:", text, re.DOTALL).group(1).strip()
academic_summary = re.search(r"Academic Summary:\s*(.*?)Work Summary:", text, re.DOTALL).group(1).strip()
work_summary = re.search(r"Work Summary:\s*(.*)", text, re.DOTALL).group(1).strip()



In [68]:
#print(work_summary)

In [69]:
# Store in appropriate data structures
profile_data = {
    "Resource Name": resource_name,
    "Job Role": job_role,
    "Percentage Match": percentage_match,
    "Missing Keywords": missing_keywords,
    "Summary": summary,
    "Key Skills & Competencies": key_skills_competencies.split("\n"),
    "Academic Summary": academic_summary.split("\n"),
    "Work Summary": []
}

# Extract individual projects in Work Summary
projects = re.findall(r"\d+\.\s(.*?)\n\s+-\sEnvironment:\s(.*?)\n\s+-\sOutline:\s(.*?)\n\s+-\sResponsibilities:\s(.*?)\n(?=\d+\.|\Z)", work_summary, re.DOTALL)
for project in projects:
    project_data = {
        "Project": project[0],
        "Environment": project[1],
        "Outline": project[2].strip(),
        "Responsibilities": project[3].strip()
    }
    profile_data["Work Summary"].append(project_data)

# Print extracted data
for key, value in profile_data.items():
    print(f"{key}: {value}")

Resource Name: Munazza Jamal
Job Role: Manual and Automation Software Tester
Percentage Match: 40%
Missing Keywords: Data Engineers, Databricks certification
Summary: A highly motivated resource with 5+ years of experience in the field of Manual and Automation software testing. Understands and possesses test management skills as well as automation scripting skills under the best tools and frameworks. Skilled in Smoke Testing, Functional Testing, Integration Testing, System Testing, User Interface testing etc. Have proven expertise in working on different types of test deliverables – Test Plans, Test Cases, Testing Strategies, Test scripts and Test summary reports. Very well acquainted with Test Management and Defect tracking systems such as JIRA and Azure DevOps. Experience in leading, mentoring and guiding the QA team. Also proficient in Test Automation tools such as Selenium WebDriver, Appium, and Cypress.
Key Skills & Competencies: ['1. Technical Expertise: Munazza has 5+ years of e

In [74]:
print(profile_data['Work Summary'][0]['Project'])

Food Delivery – Web / Mobile Application & Backend Services


### Generating Word Document
Will include everything according to the provided format except for "Percentage Match" and "Missing Keywords"